# Demo with `numpy.f2py`

> The purpose of the F2PY *–Fortran to Python interface generator–* is to provide a connection between Python and Fortran languages.
> https://numpy.org/doc/stable/f2py/

It comes with a CLI:
```bash
python -m numpy.f2py
```

and with an API:
```python
import numpy.f2py
```

The focus is on the use of the CLI here as the API is "not mature"
according the numpy's documentation.

Requirements for the examples below to work are:
- `numpy`
- a FORTRAN compiler (e.g. gfortran or ifort)

## A first example: calling simple FORTRAN subroutines

The FORTRAN source code for this example is *a_simple_call/my_file.f90*.

### Building the wrapper

There are 3 ways of building a wrapper around FORTRAN code:
1. compiling directly the pristine source code with F2PY and **let F2PY infer the interface**

2. generating and tweaking a **signature file** to teach F2PY about the code interface before compilation (if you cannot modify the source code)

3. adding **special fortran comments** in source to teach F2PY about the code interface before compilation (if you can modify the source code)

#### 1. Quick and easy

Directly compile the source code: `python -m numpy.f2py -c <fortran_code> -m <module_name>`

In [1]:
! cd a_simple_call && \
python -m numpy.f2py -c my_file.f90 -m my_package \
--quiet --f90exec=/data/conda/hj-38-nompi/bin/x86_64-conda-linux-gnu-gfortran

Could not locate executable gfortran
Could not locate executable f95


In [2]:
! ls a_simple_call

my_file.f90  my_package.cpython-38-x86_64-linux-gnu.so


In [3]:
! rm a_simple_call/my_package*.so

#### 2. With an intermediate signature file

Generate a signature file: `numpy.f2py -m <module_name> -h <signature_file> <source_files>`

In [4]:
! cd a_simple_call && \
python -m numpy.f2py -m my_package -h my_signature.pyf my_file.f90 \
--quiet --overwrite-signature

In [5]:
! ls a_simple_call

my_file.f90  my_signature.pyf


Give the signature file when compiling: `numpy.f2py -c <signature_file> <source_files>`

In [6]:
! cd a_simple_call && \
python -m numpy.f2py -c my_signature.pyf my_file.f90 \
--quiet --f90exec=/data/conda/hj-38-nompi/bin/x86_64-conda-linux-gnu-gfortran

Could not locate executable gfortran
Could not locate executable f95


#### 3. Make source code more explicit 

By e.g. using FORTRAN `intent`, if not already, and, if not enough, using F2PY special comments `!f2py ...`

### Using the package

In [7]:
import numpy

In [8]:
import a_simple_call.my_package as my_package

F2PY generates some basic documentation for the package and its functions.

In [9]:
print(my_package.__doc__)

This module 'my_package' is auto-generated with f2py (version:2).
Functions:
  the_result = add_scalar(an_array,a_scalar,n=len(an_array))
  the_result = add_array(array_one,array_two,n=len(array_one))
.


In [10]:
print(my_package.add_scalar.__doc__)

the_result = add_scalar(an_array,a_scalar,[n])

Wrapper for ``add_scalar``.

Parameters
----------
an_array : input rank-1 array('f') with bounds (n)
a_scalar : input float

Other Parameters
----------------
n : input int, optional
    Default: len(an_array)

Returns
-------
the_result : rank-1 array('f') with bounds (n)



In [11]:
my_package.add_scalar(numpy.zeros((4,)), 3.)

array([3., 3., 3., 3.], dtype=float32)

Note, F2PY does the casting for you if it can, e.g. converting default
numpy array of `dtype=numpy.float64` to `dtype=numpy.float32` (since simple
precision is required in the Fortran code).

In [12]:
my_package.add_array(1 * numpy.ones((4,)), 3 * numpy.ones((4,)))

array([4., 4., 4., 4.], dtype=float32)

F2PY does the slicing for you if it can, i.e. if `n < len(an_array)`.

In [13]:
my_package.add_scalar(numpy.zeros((4,)), 3., n=3)

array([3., 3., 3.], dtype=float32)

In [14]:
my_package.add_scalar(numpy.zeros((4,)), 3., n=5)

error: (len(an_array)>=n) failed for 1st keyword n: add_scalar:n=5

## A second example: modifying an array inplace

The FORTRAN source code for this example is *intent_inout/my_file.f90*.

In [15]:
! cd intent_inout && \
python -m numpy.f2py -c my_file.f90 -m my_package \
--quiet --f90exec=/data/conda/hj-38-nompi/bin/x86_64-conda-linux-gnu-gfortran

Could not locate executable gfortran
Could not locate executable f95


In [16]:
import intent_inout.my_package as my_package

### Importance of the data type

In [17]:
my_package.add_scalar(numpy.zeros((4,)), 3.)

ValueError: failed in converting 1st argument `an_array' of my_package.add_scalar to C/Fortran array

For variables with `intent(inout)` in the Fortran code, numpy prevents the
casting by default.

In [18]:
arr = numpy.zeros((4,), dtype=numpy.float32)
my_package.add_scalar(arr, 3.)
print(arr)

[3. 3. 3. 3.]


### Importance of the C vs FORTRAN contiguity (i.e. row-major vs. column-major)

In [19]:
arr = numpy.zeros((3, 2), dtype=numpy.float32)
my_package.add_scalar(arr, 3.)
print(arr)

ValueError: failed in converting 1st argument `an_array' of my_package.add_scalar to C/Fortran array

For variables with `intent(inout)` in the Fortran code, numpy prevents the
change in array orientation by default.

In [20]:
arr = numpy.zeros((3, 2), order='F', dtype=numpy.float32)
my_package.add_scalar(arr, 3.)
print(arr)

[[3. 3.]
 [3. 3.]
 [3. 3.]]


### Alternatives

If you know of these inconsistencies, but you don't to change neither
the numpy array nor the Fortran array type/orientation, you can use
*!f2py intent(in,out)* to return the Fortran array, rather than modify
it inplace.

In [21]:
arr = numpy.zeros((3, 2))
arr.flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

In [22]:
arr2 = my_package.add_scalar_in_out(arr, 3.)
arr2

array([[3., 3.],
       [3., 3.],
       [3., 3.]], dtype=float32)

In [23]:
arr2.flags

  C_CONTIGUOUS : False
  F_CONTIGUOUS : True
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

Or you can ask F2PY to try harder and do the casting/array re-orientation
anyway using *!f2py intent(inplace)*.

In [24]:
arr = numpy.zeros((3, 2))
arr.flags, hex(id(arr))

(  C_CONTIGUOUS : True
   F_CONTIGUOUS : False
   OWNDATA : True
   WRITEABLE : True
   ALIGNED : True
   WRITEBACKIFCOPY : False
   UPDATEIFCOPY : False,
 '0x7fdb90035620')

In [25]:
my_package.add_scalar_inplace(arr, 3.)
arr

array([[3., 3.],
       [3., 3.],
       [3., 3.]], dtype=float32)

In [26]:
arr.flags, hex(id(arr))

(  C_CONTIGUOUS : False
   F_CONTIGUOUS : True
   OWNDATA : True
   WRITEABLE : True
   ALIGNED : True
   WRITEBACKIFCOPY : False
   UPDATEIFCOPY : False,
 '0x7fdb90035620')

## A third example: allocatable arrays

The FORTRAN source code for this example is *allocatable_array/my_file.f90*.

In [27]:
! cd allocatable_array && \
python -m numpy.f2py -c my_file.f90 -m my_package \
--quiet --f90exec=/data/conda/hj-38-nompi/bin/x86_64-conda-linux-gnu-gfortran

Could not locate executable gfortran
Could not locate executable f95


In [28]:
import allocatable_array.my_package as my_package

In [29]:
print(my_package.my_module.my_array)

None


### Running this would trigger a segmentation fault

In [ ]:
my_package.my_module.add_scalar(3.)

### Allocate memory first!

In [30]:
my_package.my_module.allocate_memory(10)

In [31]:
print(my_package.my_module.my_array)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [32]:
my_package.my_module.add_scalar(3.)

In [33]:
print(my_package.my_module.my_array)

[3. 3. 3. 3. 3. 3. 3. 3. 3. 3.]


### Allocate memory using Python

In [34]:
import numpy
arr = numpy.zeros((3,), order='F', dtype=numpy.float32)
my_package.my_module.my_array = arr

In [35]:
print(my_package.my_module.my_array)

[0. 0. 0.]


In [36]:
my_package.my_module.add_scalar(3.)
print(my_package.my_module.my_array)

[3. 3. 3.]


The only drawback of the allocation in Python is that a new array is created in the process.

Indeed, the array used for the memory allocation was not modified after adding the scalar.

In [37]:
arr

array([0., 0., 0.], dtype=float32)

### Funny finding?

#### Behaviour of a view of a numpy array

An array and a view of it are two different objects (different memory locations).

In [38]:
array1 = numpy.zeros((10,))
print(array1, hex(id(array1)))

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 0x7fdb90035300


In [39]:
array2 = array1[:]
print(array2, hex(id(array2)))

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 0x7fdb90130df0


The view has a base array.

In [40]:
hex(id(array2.base))

'0x7fdb90035300'

The two objects share memory.

In [41]:
numpy.shares_memory(array1, array2)

True

Consequently, modifying one modifies the other.

In [42]:
array1[0] = 7
print(array1, array2)

[7. 0. 0. 0. 0. 0. 0. 0. 0. 0.] [7. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


#### Does retrieving the internal allocatable array returns a view of it?

Each time the array is retrieved, a new object is created, it could be a view.

In [43]:
ma1 = my_package.my_module.my_array
print(ma1, hex(id(ma1)))

[3. 3. 3.] 0x7fdb90047c60


In [44]:
ma2 = my_package.my_module.my_array
print(ma2, hex(id(ma2)))

[3. 3. 3.] 0x7fdb90047ee0


#### Do they own their own memory?

Neither is the base for the other. So neither is a view of the other.

In [45]:
(ma1.base is None) and (ma2.base is None)

True

Yet, they share memory.

In [46]:
numpy.shares_memory(ma1, ma2)

True

In fact, they point to the same data-area storing the array contents.

In [47]:
hex(ma1.__array_interface__['data'][0]), hex(ma2.__array_interface__['data'][0])

('0x55d18c4d5410', '0x55d18c4d5410')

#### Behaves like a view without being a view?

Unsurprisingly, given the facts above, modifying one array modifies all arrays.

In [48]:
my_package.my_module.my_array[0] = 9
print(ma1, ma2)

[9. 3. 3.] [9. 3. 3.]
